# Agressive Social Media Text Processing



## Project Main code
1. Read in the dataset
1. Pre-process Dataset
1. Create Labels for classification
1. Build Model 1
1. Train Model 1
1. Display Accuracy
1. Build Model 2
1. Train Model 2
1. Display Accuracy

## Prepare Google Drive and Filename

In [9]:
#All library imports are here
import numpy as np
import pandas as pd
import re
import string
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from sklearn.metrics import make_scorer, f1_score, accuracy_score, recall_score, precision_score, classification_report, precision_recall_fscore_support
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from keras.layers.core import Dense, Dropout, Activation, Lambda
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
dataset_file = 'D:/project/dataSet.csv'

## Read Dataset



In [12]:
#In this section we Read the dataset from the Paper

# Data is stored in csv format

# Load CSV using Pandas

filename = dataset_file
names = ['id', 'post', 'label']
df_original = pd.read_csv(filename, names=names, encoding='UTF-8')
print(df_original.shape)

(12000, 3)


## Print the first 10 lines of the data as an example

In [13]:
first_10_rows = df_original.head(10)
print(first_10_rows)

                            id  \
0   facebook_corpus_msr_401470   
1   facebook_corpus_msr_386695   
2   facebook_corpus_msr_373389   
3   facebook_corpus_msr_917635   
4   facebook_corpus_msr_382517   
5   facebook_corpus_msr_403274   
6  facebook_corpus_msr_1723083   
7   facebook_corpus_msr_325257   
8    facebook_corpus_msr_23447   
9  facebook_corpus_msr_1477104   

                                                post label  
0  Mahmood Ghaznavi Aor ABdali ko bhol gaya ha tu...   OAG  
1  Bhai 60sal pehle desh me kya tha pehle pta kro...   CAG  
2  chutiya friday ko isliye releae krte kyoki wee...   CAG  
3                                         जय मोदीराज   CAG  
4     UPA walo ne bahot kuch kr diya tha desh k liye   CAG  
5  Pan ko Aadhar se link ki zarurat kuy hai? Supr...   CAG  
6  काकर पाथर जोड़ के मस्जिद लई बनाय।\n\nता चढ़ि मुल...   OAG  
7  Us raat tere papa k jageh mera sperm gya tha u...   OAG  
8                                       गटर के कीड़े   OAG  
9  Waise bandhu

## Read the processed data back to a dataframe

In [14]:
dataset_processed_file = 'D:/project/DataSetProcessed.csv'

In [15]:
filename = dataset_processed_file
header = ['id', 'post', 'label']
df_processed = pd.read_csv(filename, names=header)
print(df_processed.shape)
print(df_processed.head(10))

(11999, 3)
                             id  \
0   facebook_corpus_msr_401470    
1   facebook_corpus_msr_386695    
2   facebook_corpus_msr_373389    
3   facebook_corpus_msr_917635    
4   facebook_corpus_msr_382517    
5   facebook_corpus_msr_403274    
6  facebook_corpus_msr_1723083    
7   facebook_corpus_msr_325257    
8    facebook_corpus_msr_23447    
9  facebook_corpus_msr_1477104    

                                                post label  
0   Mahmood Ghaznavi Aor ABdali ko bhol gaya ha t...   OAG  
1   Bhai 60sal pehle desh me kya tha pehle pta kr...   CAG  
2   chutiya friday ko isliye releae krte kyoki we...   CAG  
3                                       jai modiraj    CAG  
4    UPA walo ne bahot kuch kr diya tha desh k liye    CAG  
5   Pan ko Aadhar se link ki zarurat kuy hai? Sup...   CAG  
6   kaakar pathar jod ke masjid lai banaay. ta ch...   OAG  
7   Us raat tere papa k jageh mera sperm gya tha ...   OAG  
8                                    gater ke keede   

## Split the dataset into training and test data

In [16]:
df_processed = df_processed.iloc[: , 1:]  # Drop first column
print(df_processed.shape)

# Save without ID to csv file
file_no_id = 'D:/project/fileNoID.csv'
df_processed.to_csv(file_no_id, index=False, header=False)

# Decide on a percentage split 70/30
#msk = np.random.rand(len(df)) < 0.7
#train_df = df_processed[msk]
#test_df = df_processed[~msk]

#print(train_df.shape)
#print(test_df.shape)

#print(train_df.head(10))

(11999, 2)


In [17]:
file_no_id = 'D:/project/fileNoID.csv'
data_cleanedup = 'D:/project/datacleanedup.csv'

In [15]:
import torch
print(f'Can I can use GPU now? -- {torch.cuda.is_available()}')
device = torch.device('cuda')



Can I can use GPU now? -- True


In [19]:
header = ['post', 'label']
df_pretoken = pd.read_csv(data_cleanedup, header=None, names=header)
print(df_pretoken.shape)

X_pretoken = df_pretoken['post']
Y_pretoken = df_pretoken['label']

print(X_pretoken.shape)
print(Y_pretoken.shape)

tokenizer = Tokenizer(num_words=4000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,split=' ')
tokenizer.fit_on_texts(X_pretoken)
#print(tokenizer.word_index)

X = tokenizer.texts_to_sequences(X_pretoken)
#X = word_tokenize(str(X_pretoken))  # Using NLTK
#print(X)


sum_tot = 0
for seq in X:
    sum_tot += len(seq)

print(float(sum_tot/11634.0))

new_x = []
new_y = []

for i in range(0, len(X)):
    if len(X[i]) < 400:
        new_x.append(X[i])
        new_y.append(Y_pretoken[i])

print(len(new_x))
print(len(new_y))

X = pad_sequences(new_x)
Y = pd.get_dummies(new_y)

print(X.shape)
print(Y.shape)

#print(X)
#print(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)


X_train_tensor = torch.Tensor(X_train)
Y_train_tensor = torch.Tensor(np.array(Y_train))

X_train_tensor = X_train_tensor.to(device)
Y_train_tensor = Y_train_tensor.to(device)



(11788, 2)
(11788,)
(11788,)
22.9778236204229
11773
11773
(11773, 387)
(11773, 3)
(9418, 387)
(9418, 3)
(2355, 387)
(2355, 3)


Build the CNN Network

In [20]:
# Deep Neural Network "MLP": multi layer Perceptron
model = Sequential()


model.add(Dense(256, input_dim=X_train.shape[1]))

# 0.42 accuracy.
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(3))
model.add(Activation('softmax'))

# we'll use categorical xent for the loss, and RMSprop as the optimizer
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

print("Training...")
model.fit(X_train, Y_train, epochs=5, batch_size=32, validation_split=0.2)

print("Generating test predictions...")
preds = model.predict(X_test, verbose=0)
print(preds)

# Evaluating the model
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = 32)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

Training...
Epoch 1/5
236/236 [==============================] - 2s 3ms/step - loss: 54.3709 - accuracy: 0.3774 - val_loss: 2.0733 - val_accuracy: 0.4066
Epoch 2/5
236/236 [==============================] - 1s 2ms/step - loss: 2.8033 - accuracy: 0.3889 - val_loss: 1.0768 - val_accuracy: 0.4039
Epoch 3/5
236/236 [==============================] - 1s 2ms/step - loss: 1.3854 - accuracy: 0.4055 - val_loss: 1.0586 - val_accuracy: 0.4066
Epoch 4/5
236/236 [==============================] - 1s 2ms/step - loss: 1.2604 - accuracy: 0.4027 - val_loss: 1.0489 - val_accuracy: 0.4087
Epoch 5/5
236/236 [==============================] - 1s 2ms/step - loss: 1.1710 - accuracy: 0.4070 - val_loss: 1.0462 - val_accuracy: 0.4071
Generating test predictions...
[[0.3931996  0.20951858 0.39728183]
 [0.3931996  0.20951858 0.39728183]
 [0.3931996  0.20951858 0.39728183]
 ...
 [0.3931996  0.20951858 0.39728183]
 [0.3931996  0.20951858 0.39728183]
 [0.3931996  0.20951858 0.39728183]]
74/74 - 0s - loss: 1.0507 - a

In [21]:
predict = model.predict(X_test)
preds = predict

74/74 [==============================] - 0s 2ms/step


In [22]:
tmpMat = np.zeros((len(Y_test), 3), dtype=int)
for i in range(0,len(predict)):
    if(preds[i].any()==0):
        tmpMat[i][0]=1
    elif(preds[i].any()==1):
        tmpMat[i][1]=1
    else:
        tmpMat[i][2]=1

test_y = Y_test

print(precision_score(test_y, tmpMat, average='weighted'))
print(recall_score(test_y, tmpMat, average='weighted'))
print(f1_score(test_y, tmpMat, average='weighted'))  



# OF CAG
print(precision_score(test_y[:,0], tmpMat[:,0], average='weighted'))
print(recall_score(test_y[:,0], tmpMat[:,0], average='weighted'))
print(f1_score(test_y[:,0], tmpMat[:,0], average='weighted'))


# Of class NAG
print(precision_score(test_y[:,1], tmpMat[:,1], average='weighted'))
print(recall_score(test_y[:,1], tmpMat[:,1], average='weighted'))
print(f1_score(test_y[:,1], tmpMat[:,1], average='weighted'))

# Of class OAG
print(precision_score(test_y[:,2], tmpMat[:,2], average='weighted'))
print(recall_score(test_y[:,2], tmpMat[:,2], average='weighted'))
print(f1_score(test_y[:,2], tmpMat[:,2], average='weighted'))

c:\Users\nikhi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.03716463593294297
0.19278131634819534
0.06231592568321872


InvalidIndexError: (slice(None, None, None), 0)

In [23]:
#CNN
# CNN: ConvNeuralNets

nb_filter = 250
filter_length = 3
hidden_dims = 250
nb_epoch = 2

In [24]:
from keras.layers.convolutional import Convolution1D
from keras import backend as K

print('Build model...')
model = Sequential()
model.add(Embedding(4000, 128))
# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:
model.add(Convolution1D(4000, 128, 
                        activation='relu'))

def max_1d(X_train):
    return K.max(X_train, axis=1)

model.add(Lambda(max_1d, output_shape=(nb_filter,)))
model.add(Dense(hidden_dims)) 
model.add(Dropout(0.2)) 
model.add(Activation('relu'))
model.add(Dense(3))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Build model...


In [16]:
print('Train...')
model.fit(X_train, Y_train, batch_size=100, epochs=2,
          validation_split=0.2)
score, acc = model.evaluate(X_test, Y_test, batch_size=32)
print('Test score:', score)
print('Test accuracy:', acc)

Train...


NameError: name 'model' is not defined

In [ ]:
predict = model.predict(X_test)
test_y = Y_test.as_matrix()
tmpMat = np.zeros((len(Y_test), 3), dtype=int)

for i in range(0,len(predict)):
    if(predict[i][0] > predict[i][1] and predict[i][0] > predict[i][2]):
        tmpMat[i][0]=1
    elif(predict[i][1] > predict[i][0] and predict[i][1] > predict[i][2]):
        tmpMat[i][1]=1
    else:
        tmpMat[i][2]=1
        
print(precision_score(test_y, tmpMat, average='weighted'))
print(recall_score(test_y, tmpMat, average='weighted'))
print(f1_score(test_y, tmpMat, average='weighted'))

print(precision_score(test_y[:,0], tmpMat[:,0], average='weighted'))
print(recall_score(test_y[:,0], tmpMat[:,0], average='weighted'))
print(f1_score(test_y[:,0], tmpMat[:,0], average='weighted'))

print(precision_score(test_y[:,1], tmpMat[:,1], average='weighted'))
print(recall_score(test_y[:,1], tmpMat[:,1], average='weighted'))
print(f1_score(test_y[:,1], tmpMat[:,1], average='weighted'))

print(precision_score(test_y[:,2], tmpMat[:,2], average='weighted'))
print(recall_score(test_y[:,2], tmpMat[:,2], average='weighted'))
print(f1_score(test_y[:,2], tmpMat[:,2], average='weighted'))

## LSTM

In [ ]:
# LSTM Model for prediction
embed_dim = 128
lstm_out = 196
batch_size = 100

model = Sequential()
model.add(Embedding(4000, embed_dim,input_length = X.shape[1], dropout = 0.2))
model.add(LSTM(lstm_out, dropout_U = 0.2, dropout_W = 0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

In [ ]:
model.fit(X_train, Y_train, batch_size = batch_size, epochs = 2, verbose = 2)

In [ ]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

In [ ]:
p = model.predict(X_test)

In [ ]:
tmpMat = np.zeros((len(Y_test), 3), dtype=int)

for i in range(0,len(predict)):
    if(p[i][0] > p[i][1] and p[i][0] > p[i][2]):
        tmpMat[i][0]=1
    elif(p[i][1] > p[i][0] and p[i][1] > p[i][2]):
        tmpMat[i][1]=1
    else:
        tmpMat[i][2]=1

test_y = Y_test.as_matrix()

# For LSTM model
print(precision_score(test_y, tmpMat, average='weighted'))
print(recall_score(test_y, tmpMat, average='weighted'))
print(f1_score(test_y, tmpMat, average='weighted'))

# Of Class CAG
print(precision_score(test_y[:,0], tmpMat[:,0], average='weighted'))
print(recall_score(test_y[:,0], tmpMat[:,0], average='weighted'))
print(f1_score(test_y[:,0], tmpMat[:,0], average='weighted'))

# Of class NAG
print(precision_score(test_y[:,1], tmpMat[:,1], average='weighted'))
print(recall_score(test_y[:,1], tmpMat[:,1], average='weighted'))
print(f1_score(test_y[:,1], tmpMat[:,1], average='weighted'))

# Of class OAG
print(precision_score(test_y[:,2], tmpMat[:,2], average='weighted'))
print(recall_score(test_y[:,2], tmpMat[:,2], average='weighted'))
print(f1_score(test_y[:,2], tmpMat[:,2], average='weighted'))

## Naive Bayes

In [ ]:
#Naive Bayes

## SVM

## Transfer Learning